# Data Challenge - Salary Dataset (Alan Au)

### Assignment Description:

It’s helpful if we can suggest an approximate salary to job seekers for a given job post. Unfortunately, not all job postings designate the salary. This is where you come in: Your first task is to develop a salary prediction system.

The goal: provide estimated salaries for a new job posting.


### Data supplied:

You are given three CSV (comma--separated) data files:

* train_features_DATE.csv: Each row represents metadata for an individual job posting. The “jobId” column represents a unique identifier for the job posting. The remaining columns describe features of the job posting.
* train_salaries_DATE.csv: Each row associates a “jobId” with a “salary”.
* test_features_DATE.csv: Similar to train_features_DATE.csv, each row represents metadata for an individual job posting.

The first row of each file contains headers for the columns. Keep in mind that the metadata and
salary data has been extracted by our aggregation and parsing systems. As such, it’s possible
that the data is dirty (may contain errors).

### The task

You must build a model to predict the salaries for the job postings contained in test_features_DATE.csv. The output of your system should be a CSV file entitled **test_salaries.csv** where each row has the following format: jobId, salary
As a reference, your output should mirror the format of train_salaries_DATE.csv.

# Alan's Notes

* Import packages and files
* Inspect files
* Summarize and visualize data
* Choose model (probably a Random Forest Regressor)
* Train/test split on training data? (not needed for Random Forest)
* Evaluate model (out of bag)
* Get feature importance listing
* Run test data
* Cleanup and presentation

In [1]:
#!/usr/bin/python3
__author__ = 'Alan Au'
__date__   = '2018-04-11'

import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [15,5] #make the plots bigger and easier to read
#import seaborn as sns

#from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
#from sklearn.pipeline import Pipeline

#seed = int(random.random()*10000) #generate a random number between 0 and 10000
seed = 7979 #for consistency during testing, just keep the same pseudorandomly generated seed
print("Pseudorandom seed is:",seed)

Pseudorandom seed is: 7979


In [2]:
data_loc = './Salaries/' #path to my data file subdirectory; must end with '/'

# pre-specified
f_train_x = data_loc + 'test_features_2013-03-07.csv'
f_train_y = data_loc + 'train_salaries_2013-03-07.csv'
f_test_x = data_loc + 'test_features_2013-03-07.csv'

# generate this
f_test_y = data_loc + 'test_salaries.csv'

# for reference
headers_x = ['jobId','companyId','jobType','degree','major','industry','yearsExperience','milesFromMetropolis']
headers_y = ['jobId','salary']

# load data
# Treat 'NONE' as missing data for now--if it's meaningful, can re-import as-is.
explore_x_df = pd.read_csv(f_train_x, header='infer', na_values='NONE')
explore_y_df = pd.read_csv(f_train_y, header='infer', na_values='NONE')

# Check for missing/'NONE' values.
explore_x_df.count()

jobId                  1000000
companyId              1000000
jobType                1000000
degree                  762533
major                   465932
industry               1000000
yearsExperience        1000000
milesFromMetropolis    1000000
dtype: int64

In [3]:
# this time, keep the 'NONE' values
train_x_df = pd.read_csv(f_train_x, header='infer')
train_y_df = pd.read_csv(f_train_y, header='infer')
test_x_df = pd.read_csv(f_test_x, header='infer')

# Check a few random rows to make sure I loaded this correctly
train_x_df.sample(10)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
2670,JOB1362685410357,COMP48,SENIOR,NONE,NONE,EDUCATION,8,94
586714,JOB1362685994401,COMP47,VICE_PRESIDENT,BACHELORS,LITERATURE,HEALTH,19,95
683694,JOB1362686091381,COMP2,JANITOR,NONE,NONE,EDUCATION,14,66
60359,JOB1362685468046,COMP61,VICE_PRESIDENT,HIGH_SCHOOL,NONE,AUTO,4,85
51183,JOB1362685458870,COMP50,CTO,BACHELORS,ENGINEERING,AUTO,11,46
256723,JOB1362685664410,COMP43,JANITOR,HIGH_SCHOOL,NONE,HEALTH,12,6
39612,JOB1362685447299,COMP11,SENIOR,DOCTORAL,NONE,AUTO,24,27
944700,JOB1362686352387,COMP7,JANITOR,HIGH_SCHOOL,NONE,WEB,15,60
980094,JOB1362686387781,COMP58,CFO,HIGH_SCHOOL,NONE,HEALTH,2,84
306061,JOB1362685713748,COMP35,JANITOR,NONE,NONE,SERVICE,11,42


In [4]:
# Verify count for outputs
train_y_df.count()

jobId     1000000
salary    1000000
dtype: int64

# Basic Data Exploration

In [ ]:
# What are all of the companies?
companies = train_x_df['companyId']
companies.value_counts()

In [ ]:
# What are all of the job types?
jobTypes = train_x_df['jobType']
jobTypes.value_counts()

In [ ]:
# What are all of the degrees?
degrees = train_x_df['degree']
degrees.value_counts()

In [ ]:
# What are all of the majors?
majors = train_x_df['major']
majors.value_counts()

In [ ]:
# What are all of the industries?
industries = train_x_df['industry']
industries.value_counts()

In [ ]:
# What's the range of years experience?
train_x_df.sort_values(by=['yearsExperience'])
years = train_x_df['yearsExperience']

# List the yearsExperience and their count
years.value_counts().sort_index()

#plt.hist(years,bins=25); # Can plot this if helpful

In [ ]:
# What's the range of miles from metropolis?
dists = train_x_df['milesFromMetropolis']
dists.value_counts().sort_index()

#plt.hist(dists,bins=100); # Can plot this if helpful

# Secondary Data Exploration

In [ ]:
# What's my distribution of jobTypes without a major?
jobTypes_no_maj = train_x_df[train_x_df['major'] == 'NONE']['jobType']
jobTypes_no_maj.value_counts()

In [ ]:
# What's my distribution of jobTypes without a degree?
jobTypes_no_deg = train_x_df[train_x_df['degree'] == 'NONE']['jobType']
jobTypes_no_deg.value_counts()

# Salary (Outcome) Data

In [ ]:
# What's the range of salaries?
salaries = set(train_y_df['salary'])
print("Salary range:", min(salaries), max(salaries))
num_bins = max(salaries) - min(salaries)

# What's the distribution of salary data?
plt.hist(sorted(train_y_df['salary']), bins=num_bins);
plt.xlabel('Salary ($k)', fontsize = '20');
plt.ylabel('Number of listings', fontsize = '20');

In [ ]:
# Any 0-salary listings in our training set?
train_y_df[train_y_df['salary']<=0].count()

In [ ]:
# What about really low-salary listings in our training set?
train_y_df[train_y_df['salary']<=5].count()  # Pick some low threshold, like 5k annual salary.

In [ ]:
# Probably bad data, so let's take a look.
train_x_df[train_y_df['salary']<=5]

In [ ]:
# Sanity check; what's my distribution of salaries without a degree?
salary_no_deg = train_y_df[train_x_df['degree'] == 'NONE']['salary']
num_bins = max(salary_no_deg) - min(salary_no_deg)

plt.hist(sorted(salary_no_deg), bins=num_bins); #based on salary range
plt.xlabel('Salary without degree ($k)', fontsize = '20');
plt.ylabel('Number of listings', fontsize = '20');

In [ ]:
# Sanity check; what's my distribution of salaries without a major?
salary_no_maj = train_y_df[train_x_df['major'] == 'NONE']['salary']
num_bins = max(salary_no_maj) - min(salary_no_maj)

plt.hist(sorted(salary_no_maj), bins=num_bins); #based on salary range
plt.xlabel('Salary without major ($k)', fontsize = '20');
plt.ylabel('Number of listings', fontsize = '20');

## Data Exploration Notes

The feature data contains 'NONE' values. In cases where the degree is "High School" then there's no corresponding major, which is expected. 

Currently, it's about 24% of the 'degree' data and 54% of the 'major' data. It's a large enough percentage that I don't want to drop it, it appears to be distributed across the dataset, and if I use a tree-based model, it may end being useful. But if the model performs badly, I may re-evaluate.

On the other hand, 0-salary results are not useful, and there are so few that it's probably safe to drop them.

# Model Building

I'm going to start with a Random Forest Regression model.

In [5]:
# Keep data where salary is in a reasonable range (i.e. >5). Also keep 'NONE' values.
clean_y_df = train_y_df[train_y_df['salary']>=5]
clean_x_df = train_x_df[train_y_df['salary']>=5]
clean_x_df.count()

jobId                  999995
companyId              999995
jobType                999995
degree                 999995
major                  999995
industry               999995
yearsExperience        999995
milesFromMetropolis    999995
dtype: int64

In [6]:
# Build a model dataframe; note that I'm excluding the jobId
categorical = ['companyId','jobType','degree','major','industry']
continuous = ['yearsExperience','milesFromMetropolis']

model_df = pd.DataFrame()
test_df = pd.DataFrame()

for cat in list(categorical):
    model_df = pd.concat([model_df,pd.get_dummies(clean_x_df[cat])],axis=1)
    test_df = pd.concat([test_df,pd.get_dummies(test_x_df[cat])],axis=1)
for cat in list(continuous):
    model_df = pd.concat([model_df,clean_x_df[cat]],axis=1)
    test_df = pd.concat([test_df,test_x_df[cat]],axis=1)

model_df.sample(5)

,COMP0,COMP1,COMP10,COMP11,COMP12,COMP13,COMP14,COMP15,COMP16,COMP17,...,PHYSICS,AUTO,EDUCATION,FINANCE,HEALTH,OIL,SERVICE,WEB,yearsExperience,milesFromMetropolis
22491,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,21,35
239992,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,61
240128,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,12,16
715459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,23,77
828313,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,5,41


In [7]:
# Remove the jobId, which doesn't provide useful information
model_y = clean_y_df.drop(['jobId'],axis=1)['salary']
model_y.head(5)

0    130
1    101
2    137
3    142
4    163
Name: salary, dtype: int64

In [ ]:
# Hyperparamters
max_depth = None
max_trees = 40

regr_rf = RandomForestRegressor(n_estimators = max_trees, 
                                max_depth=max_depth, random_state=seed, 
                                oob_score=True, warm_start=False)
regr_rf.fit(model_df, model_y)

# For reporting
importances = regr_rf.feature_importances_
oob = regr_rf.oob_score_ #Interpret like an R2 score

print("Feature names:", list(model_df))
print("Feature importance:",importances)
print("OOB score is:",oob)